# Completing actor data

In [1]:
from src.data.transform_data import* 
from src.models.movie_success_model import* 
from src.models.actor_success_model import*
from webscraping.webscraper import*

## 1) Load actor data from csv

In [2]:
actor_df = pd.read_csv('data/actor_data.csv', index_col=0)
actor_df

,Actor date of birth,Actor gender,Actor height,Actor ethnicity,Actor age at movie release,Actor Score Index
Actor name,,,,,,
denzel_washington,1954-12-28,M,1.84,/m/0x67,26.0,10.000000
matt_damon,1970-10-08,M,1.78,/m/0cnvdq1,18.0,9.912690
tom_hanks,1956-07-09,M,1.83,/m/02p4q5p,27.0,9.741555
eddie_murphy,1961-04-03,M,1.75,/m/0x67,21.0,9.415060
tom_cruise,1962-07-03,M,1.70,/m/02ctzb,18.0,9.362255
...,...,...,...,...,...,...
j._kenneth_campbell,1947-07-22,M,NaN,NaN,43.0,0.714754
evelyn_keyes,1916-11-20,F,1.63,NaN,72.0,0.700949
james_dixon,NaN,M,NaN,NaN,NaN,0.700949


## 2) Set sample size

In [8]:
test_sample = actor_df
test_sample = test_sample.dropna(subset='Actor height')

## 3) Run the scraper

In [9]:
spider = ActorScraper(test_sample)
test_sample = spider.run_scraping()
test_sample

/Users/aiden/Desktop/EPFL/Master/M - ADA/ada-2024-project-sigma-squad/webscraping/webscraper.py:314: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.actor_data.loc[:, 'Gender'] = None
/Users/aiden/Desktop/EPFL/Master/M - ADA/ada-2024-project-sigma-squad/webscraping/webscraper.py:315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.actor_data.loc[:, 'University'] = None
/Users/aiden/Desktop/EPFL/Master/M - ADA/ada-2024-project-sigma-squad/webscraping/webscraper.py:316: SettingWithCopyWarning: 
A value 

Error fetching data for Jackie Chan: 'NoneType' object has no attribute 'strip'


  3%|▎         | 170/6539 [07:39<4:47:02,  2.70s/it]

Failed to fetch data for Ric Govea: HTTP 404


  4%|▎         | 241/6539 [10:33<5:02:58,  2.89s/it]

...

Error fetching data for Ilia Kulik: expected string or bytes-like object, got 'NoneType'


 73%|███████▎  | 4741/6539 [3:16:03<1:19:26,  2.65s/it]

Failed to fetch data for Stephi Lineburg: HTTP 404


 73%|███████▎  | 4744/6539 [3:16:14<1:14:15,  2.48s/it]


KeyboardInterrupt: 

In [10]:
test_sample.to_csv('data/actor_data_scraped.csv')